# Loading MNIST data

In [50]:
import tensorflow as tf
#load mnist data

from tensorflow.examples.tutorials.mnist import input_data
#one_hot=True represents the 10 categories (0-9) as a 10-dimensional vector 
#with value 1 at the position of the corresponding category
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Run function for every model

In [51]:
def run(train_step, summary_op, mnist, path='linear', iterations=1000, dropout=False):
    with tf.Session() as sess:
        #variables have to be initialized before training
        sess.run(tf.global_variables_initializer())

        #fileWriters for logging and plotting in tensorboard
        #we need two writers to compare training and testing in one plot
        #training writer
        writer_tr = tf.summary.FileWriter(path + '_tr', graph=sess.graph)
        #testing writer
        writer_t = tf.summary.FileWriter(path + '_t', graph=sess.graph)

        #train the model

        #because tensorflow knows the whole model as computational graph
        #it is able to use automatic differentation to find the gradients
        #when train_step is ran it will apply the gradient descent updates

        batch_size = 100

        #do iterations steps
        for i in range(iterations):
            batch = mnist.train.next_batch(batch_size)
            
            feed_dict_tr ={x: batch[0], y_: batch[1]}
            if dropout:
                feed_dict_tr[keep_prob] = 0.5
            sess.run(train_step, feed_dict=feed_dict_tr)
            
            #logging step
            if i % 100 == 0:
                #train accuracy and loss
                feed_dict_tr ={x: batch[0], y_: batch[1]}
                #we have to specify if we want to calculate dropout as a param
                #if it doesn't match the model we get an error
                #when the output is calculated (while testing) dropout deactivated
                #tensorflow scales up the input of a neuron at training time
                #original definition is to scale down the inputs at test time
                #shown to result in just a small difference, but easier to implement
                if dropout:
                    feed_dict_tr[keep_prob] = 1.0
                train_acc, train_loss, train_summ = 
                    sess.run([accuracy, cross_entropy, summary_op], 
                                 feed_dict=feed_dict_tr)
                writer_tr.add_summary(train_summ, i)
                
                print('step %d, training accuracy %g' % (i, train_acc))
                
                #test accuracy
                feed_dict_eval ={x: mnist.test.images, y_: mnist.test.labels}
                if dropout:
                    feed_dict_eval[keep_prob] = 1.0
                test_acc, test_loss, test_summ = 
                    sess.run([accuracy, cross_entropy, summary_op], 
                                 feed_dict=feed_dict_eval)
                writer_t.add_summary(test_summ, i)
                
                print('step %d, test accuracy %g' % (i, test_acc))
            
        #final accuracy, again without dropout
        feed_dict_eval ={x: mnist.test.images, y_: mnist.test.labels}
        if dropout:
            feed_dict_eval[keep_prob] = 1.0
        print('test accuracy %g' % accuracy.eval(feed_dict=feed_dict_eval))
        
        writer_tr.close()
        writer_t.close()

# Linear Model

In [52]:
tf.reset_default_graph()
#softmax regression model with single linear layer

#tensorflow does its computations on an c++ backend 
#which is much faster than python 
#and allows for additional optimizations
#python is used for declaring 
#the so called computational graph 
#(which is built and executed in c++)

#placeholders represent variables that get input when running the computation
#mnist consists of images with 28x28 pixels 
#which are flattened to 28*28=784 pixels/dimensions
#None corresponds to flexibel: in this case the batch size

#shape argument is optional but allows for catching bugs
x = tf.placeholder(tf.float32, shape=[None, 784])
#last dimension of the shape represents the 10 categories
y_ = tf.placeholder(tf.float32, shape=[None, 10])

W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
#regression model
y = tf.matmul(x,W) - b

#cross entropy as loss function
#logits means that the values arent probabilites
#applies softmax on the logits
cross_entropy = tf.reduce_mean(
                    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

#evaluate the model
correct_prediction = tf.equal(tf.argmax(y,axis=1), tf.argmax(y_,axis=1))
#correct_prediction is a list of booleans 
#and converting them to numerics allows calculating the average accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#define a optimization algorithm
#standard gradient descent -> we want to minize the loss (cross_entropy)
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
    
#we want to log the loss and accuracy of the model
loss = tf.summary.scalar('loss', cross_entropy)
acc = tf.summary.scalar('accuracy', accuracy)
#we merge them to write them to a summary_writer simultaneously
summary_op = tf.summary.merge_all()

In [53]:
run(train_step, summary_op, mnist, iterations=10000)

step 0, training accuracy 0.25
step 0, test accuracy 0.1319
step 100, training accuracy 0.9
step 100, test accuracy 0.8938
step 200, training accuracy 0.97
step 200, test accuracy 0.906
step 300, training accuracy 0.92
step 300, test accuracy 0.9036
step 400, training accuracy 0.95
step 400, test accuracy 0.9117
step 500, training accuracy 0.91
step 500, test accuracy 0.911
step 600, training accuracy 0.95
step 600, test accuracy 0.9133
step 700, training accuracy 0.91
step 700, test accuracy 0.9146
step 800, training accuracy 0.97
step 800, test accuracy 0.9194
step 900, training accuracy 0.91
step 900, test accuracy 0.9184
step 1000, training accuracy 0.94
step 1000, test accuracy 0.9193
step 1100, training accuracy 0.94
step 1100, test accuracy 0.9207
step 1200, training accuracy 0.95
step 1200, test accuracy 0.9233
step 1300, training accuracy 0.94
step 1300, test accuracy 0.9164
step 1400, training accuracy 0.97
step 1400, test accuracy 0.919
step 1500, training accuracy 0.96
step

# Initalization functions for further model

In [54]:
#initialization of the weights and bias with small amount of noise
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

#we are using relu so its good practice to initialize bias with small positive value
def bias_variable(shape):
  initial = tf.constant(-0.1, shape=shape)
  return tf.Variable(initial)

# MLP with 3 hidden layers without dropout

In [55]:
def mlp(dropout=False):
    #is not depending on the data and can be choosen freely
    n_neurons1 = 1500
    n_neurons2 = 1500
    n_neurons3 = 1500

    W_fc1 = weight_variable([784, n_neurons1])
    b_fc1 = bias_variable([n_neurons1])
    h_fc1 = tf.nn.relu(tf.matmul(x, W_fc1) - b_fc1)
    
    if dropout:
        #dropout layer
        #dropout with probability of keep_prob 
        #applied to neurons of the densely connected layer
        h_fc1 = tf.nn.dropout(h_fc1, keep_prob)

    W_fc2 = weight_variable([n_neurons1, n_neurons2])
    b_fc2 = bias_variable([n_neurons2])
    h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) - b_fc2)
    
    if dropout:
        h_fc2 = tf.nn.dropout(h_fc2, keep_prob)

    W_fc3 = weight_variable([n_neurons2, n_neurons3])
    b_fc3 = bias_variable([n_neurons3])
    h_fc3 = tf.nn.relu(tf.matmul(h_fc2, W_fc3) - b_fc3)
    
    if dropout:
        h_fc3 = tf.nn.dropout(h_fc3, keep_prob)

    W_out = weight_variable([n_neurons3, 10])
    b_out = bias_variable([10])

    #Wx+b which can then be fed into softmax
    y_mlp = tf.matmul(h_fc3, W_out) - b_out
    return y_mlp

In [56]:
tf.reset_default_graph()

#shape argument is optional but allows for catching bugs
x = tf.placeholder(tf.float32, shape=[None, 784])
#last dimension of the shape represents the 10 categories
y_ = tf.placeholder(tf.float32, shape=[None, 10])
#placeholder for the probability of keeping a neurons output
keep_prob = tf.placeholder(tf.float32)

y_mlp_wo_dropout = mlp()

#loss function like in regression model 
#but with the correct logits (from the conv net)
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_mlp_wo_dropout))

#more sophisticated adam optimizer (incorporated learning rate and momentum)
train_step = tf.train.AdamOptimizer().minimize(cross_entropy)

#calculate the accuracy like in regression model
correct_prediction = tf.equal(tf.argmax(y_mlp_wo_dropout, axis=1), 
                                  tf.argmax(y_, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

tf.summary.scalar('loss', cross_entropy)
tf.summary.scalar('accuracy', accuracy)
summary_op = tf.summary.merge_all()

In [57]:
run(train_step, summary_op, mnist, path='mlp_wo_dropout', iterations=20000)

step 0, training accuracy 0.37
step 0, test accuracy 0.2166
step 100, training accuracy 0.95
step 100, test accuracy 0.9237
step 200, training accuracy 0.96
step 200, test accuracy 0.9367
step 300, training accuracy 0.97
step 300, test accuracy 0.9459
step 400, training accuracy 0.98
step 400, test accuracy 0.9543
step 500, training accuracy 1
step 500, test accuracy 0.9602
step 600, training accuracy 0.99
step 600, test accuracy 0.9588
step 700, training accuracy 0.99
step 700, test accuracy 0.9545
step 800, training accuracy 0.99
step 800, test accuracy 0.9595
step 900, training accuracy 1
step 900, test accuracy 0.9622
step 1000, training accuracy 1
step 1000, test accuracy 0.968
step 1100, training accuracy 0.99
step 1100, test accuracy 0.9739
step 1200, training accuracy 1
step 1200, test accuracy 0.9718
step 1300, training accuracy 0.99
step 1300, test accuracy 0.9647
step 1400, training accuracy 0.99
step 1400, test accuracy 0.9708
step 1500, training accuracy 1
step 1500, test 

step 12900, training accuracy 0.99
step 12900, test accuracy 0.9775
step 13000, training accuracy 1
step 13000, test accuracy 0.9799
step 13100, training accuracy 1
step 13100, test accuracy 0.9792
step 13200, training accuracy 1
step 13200, test accuracy 0.9763
step 13300, training accuracy 1
step 13300, test accuracy 0.9803
step 13400, training accuracy 1
step 13400, test accuracy 0.9735
step 13500, training accuracy 1
step 13500, test accuracy 0.9755
step 13600, training accuracy 1
step 13600, test accuracy 0.976
step 13700, training accuracy 1
step 13700, test accuracy 0.9793
step 13800, training accuracy 1
step 13800, test accuracy 0.9809
step 13900, training accuracy 1
step 13900, test accuracy 0.9775
step 14000, training accuracy 1
step 14000, test accuracy 0.978
step 14100, training accuracy 1
step 14100, test accuracy 0.9785
step 14200, training accuracy 1
step 14200, test accuracy 0.9781
step 14300, training accuracy 1
step 14300, test accuracy 0.9791
step 14400, training acc

# MLP with 3 hidden layers with dropout

In [58]:
tf.reset_default_graph()

#shape argument is optional but allows for catching bugs
x = tf.placeholder(tf.float32, shape=[None, 784])
#last dimension of the shape represents the 10 categories
y_ = tf.placeholder(tf.float32, shape=[None, 10])
#placeholder for the probability of keeping a neurons output
keep_prob = tf.placeholder(tf.float32)

y_mlp_dropout = mlp(dropout=True)

#loss function like in regression model 
#but with the correct logits (from the conv net)
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_mlp_dropout))

#more sophisticated adam optimizer (incorporates learning rate and momentum)
train_step = tf.train.AdamOptimizer().minimize(cross_entropy)

#calculate the accuracy like in regression model
correct_prediction = tf.equal(tf.argmax(y_mlp_dropout, axis=1), 
                                  tf.argmax(y_, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

tf.summary.scalar('loss', cross_entropy)
tf.summary.scalar('accuracy', accuracy)
summary_op = tf.summary.merge_all()

In [59]:
run(train_step, summary_op, mnist, path='mlp_dropout', iterations=20000, dropout=True)

step 0, training accuracy 0.22
step 0, test accuracy 0.1269
step 100, training accuracy 0.92
step 100, test accuracy 0.8944
step 200, training accuracy 0.95
step 200, test accuracy 0.9261
step 300, training accuracy 0.96
step 300, test accuracy 0.9323
step 400, training accuracy 0.95
step 400, test accuracy 0.9361
step 500, training accuracy 0.96
step 500, test accuracy 0.9393
step 600, training accuracy 0.93
step 600, test accuracy 0.9462
step 700, training accuracy 0.98
step 700, test accuracy 0.9476
step 800, training accuracy 0.94
step 800, test accuracy 0.9472
step 900, training accuracy 0.93
step 900, test accuracy 0.9506
step 1000, training accuracy 0.96
step 1000, test accuracy 0.9525
step 1100, training accuracy 0.95
step 1100, test accuracy 0.9531
step 1200, training accuracy 0.92
step 1200, test accuracy 0.9542
step 1300, training accuracy 0.95
step 1300, test accuracy 0.9583
step 1400, training accuracy 0.94
step 1400, test accuracy 0.9584
step 1500, training accuracy 0.94


step 12600, training accuracy 0.98
step 12600, test accuracy 0.9819
step 12700, training accuracy 0.99
step 12700, test accuracy 0.9819
step 12800, training accuracy 1
step 12800, test accuracy 0.9814
step 12900, training accuracy 0.97
step 12900, test accuracy 0.9795
step 13000, training accuracy 0.99
step 13000, test accuracy 0.9801
step 13100, training accuracy 1
step 13100, test accuracy 0.9806
step 13200, training accuracy 1
step 13200, test accuracy 0.9807
step 13300, training accuracy 0.97
step 13300, test accuracy 0.9802
step 13400, training accuracy 0.99
step 13400, test accuracy 0.9789
step 13500, training accuracy 1
step 13500, test accuracy 0.9809
step 13600, training accuracy 0.99
step 13600, test accuracy 0.9804
step 13700, training accuracy 1
step 13700, test accuracy 0.9816
step 13800, training accuracy 0.99
step 13800, test accuracy 0.981
step 13900, training accuracy 1
step 13900, test accuracy 0.982
step 14000, training accuracy 0.99
step 14000, test accuracy 0.9815


# Convolutional Neural Network

In [60]:
#convolution with stride 1 and zero padding -> input size = output size
#size specified as [batch, height, width, channels] <- standard
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

#max pooling with 2x2 window
def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [61]:
#multilayer convolutional network
tf.reset_default_graph()

#shape argument is optional but allows for catching bugs
x = tf.placeholder(tf.float32, shape=[None, 784])
#last dimension of the shape represents the 10 categories
y_ = tf.placeholder(tf.float32, shape=[None, 10])

#first convolutional layer
n_output_channel_conv1 = 32

#convolution will produce 5x5 patches in 1 channel and 32 output channels (features)
W_conv1 = weight_variable([5, 5, 1, n_output_channel_conv1])
#add bias for each output channel
b_conv1 = bias_variable([n_output_channel_conv1])

#to apply the layer we have to reshape the image (from flat into a 4d tensor)
#remember: the data format is specified as [batch, height, width, channels]
x_image = tf.reshape(x, [-1, 28, 28, 1]) #-1 corresponds to inferring the dimension

#convolve image with the weights, add bias and apply relu activation
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
#maxpooling reduces size to 14x14 -> because we take a 2x2 window
h_pool1 = max_pool_2x2(h_conv1)

#second convolutional layer
n_output_channel_conv2 = 64

#again 5x5 patches but from the first layer we get 32 channels 
#and we produce 64 output channels
W_conv2 = weight_variable([5, 5, n_output_channel_conv1, n_output_channel_conv2])
b_conv2 = bias_variable([n_output_channel_conv2])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) - b_conv2)
#maxpooling reduces size to 7x7
h_pool2 = max_pool_2x2(h_conv2)

#output layer

#from the second convolutional layer we get 64 features with size 7x7
W_fc2 = weight_variable([7 * 7 * n_output_channel_conv2, 10])
b_fc2 = bias_variable([10])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*n_output_channel_conv2])
y_conv = tf.matmul(h_pool2_flat, W_fc2) - b_fc2

#loss function
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

train_step = tf.train.AdamOptimizer().minimize(cross_entropy)

#calculate the accuracy like in regression model
correct_prediction = tf.equal(tf.argmax(y_conv, axis=1), tf.argmax(y_, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

tf.summary.scalar('loss', cross_entropy)
tf.summary.scalar('accuracy', accuracy)
summary_op = tf.summary.merge_all()

In [62]:
run(train_step, summary_op, mnist, path='conv', iterations=20000, dropout=False)

step 0, training accuracy 0.08
step 0, test accuracy 0.0718
step 100, training accuracy 0.88
step 100, test accuracy 0.9234
step 200, training accuracy 0.94
step 200, test accuracy 0.9529
step 300, training accuracy 0.96
step 300, test accuracy 0.9579
step 400, training accuracy 0.99
step 400, test accuracy 0.9722
step 500, training accuracy 0.97
step 500, test accuracy 0.9765
step 600, training accuracy 0.98
step 600, test accuracy 0.9769
step 700, training accuracy 0.99
step 700, test accuracy 0.9781
step 800, training accuracy 0.97
step 800, test accuracy 0.9824
step 900, training accuracy 0.97
step 900, test accuracy 0.9829
step 1000, training accuracy 1
step 1000, test accuracy 0.9821
step 1100, training accuracy 0.99
step 1100, test accuracy 0.9814
step 1200, training accuracy 0.98
step 1200, test accuracy 0.9846
step 1300, training accuracy 0.99
step 1300, test accuracy 0.9831
step 1400, training accuracy 1
step 1400, test accuracy 0.9841
step 1500, training accuracy 0.98
step 1

step 12800, test accuracy 0.9933
step 12900, training accuracy 1
step 12900, test accuracy 0.9932
step 13000, training accuracy 1
step 13000, test accuracy 0.9937
step 13100, training accuracy 1
step 13100, test accuracy 0.9938
step 13200, training accuracy 1
step 13200, test accuracy 0.9943
step 13300, training accuracy 1
step 13300, test accuracy 0.9936
step 13400, training accuracy 1
step 13400, test accuracy 0.99
step 13500, training accuracy 1
step 13500, test accuracy 0.9917
step 13600, training accuracy 1
step 13600, test accuracy 0.9891
step 13700, training accuracy 1
step 13700, test accuracy 0.9855
step 13800, training accuracy 1
step 13800, test accuracy 0.9861
step 13900, training accuracy 1
step 13900, test accuracy 0.9904
step 14000, training accuracy 1
step 14000, test accuracy 0.9896
step 14100, training accuracy 1
step 14100, test accuracy 0.9918
step 14200, training accuracy 1
step 14200, test accuracy 0.9932
step 14300, training accuracy 0.99
step 14300, test accurac

![Accuracies](tensorboard_plot.png)

# PyTorch version of the CNN

In [1]:
# MNIST CNN classifier 
# Code adopted from GunhoChoi

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.nn.init as init

## Data and hyperparameter

In [2]:
# hyperparameters
train_batch_size = 100

# Download Data
mnist_train = datasets.MNIST("./", train=True, transform=transforms.ToTensor(), 
                                 target_transform=None, download=True)
mnist_test  = datasets.MNIST("./", train=False, transform=transforms.ToTensor(), 
                                 target_transform=None, download=True)

# Set Data Loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=mnist_train, 
                                               batch_size=train_batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=mnist_test, 
                                               batch_size=mnist_test.__len__(), shuffle=False)

Processing...
Done!


## Model

In [3]:
class CNN(nn.Module):
    def __init__(self):
        # this method just defines some attributes (maybe layers)
        # computation happens in forward method
    
        super().__init__()

        # channels/filter of first and second conv layer
        self.out_channels1 = 32
        self.out_channels2 = 64 

        # first conv layer (second analog)
        # 1 input channel (mnist is greyscale)
        # specified output channels
        # kernel_size is 5x5 (symmetric if just one value specified, tuples possible)
        # stride is 1x1 (like above)
        # padded with 2 zeros at each dim
        # if want same width and length of this image after conv2d: padding=(kernel_size-1)/2 if stride=1
        self.conv1 = nn.Conv2d(1, self.out_channels1, kernel_size=5, 
                                   stride=1, padding=2)  
        self.conv2 = nn.Conv2d(self.out_channels1, self.out_channels2, kernel_size=5, 
                                   stride=1, padding=2)

        # output layer (we will get 7x7x out_channels2 inputs)
        # output to 10 dims (onehot from digits 0 to 9)
        # linear adds the bias 
        self.fc1 = nn.Linear(7*7*self.out_channels2, 10)
        
        self._initialize_weights()

    def forward(self, x):
        # instead of using just layers as attributed 
        # we also use the functional interface for relu and max_pool2d
        
        # first we apply the conv layer, then relu activation to the outputs
        # we max_pool2d with size 2x2 (again one digits means symmetric)
        # stride is at default set to fit the pooling size (no overlap)
        x = F.max_pool2d(F.relu(self.conv1(x)), 2) 
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        # reshape to batch_size (is in dim 0) 
        # and dim that matches (is 7 x 7 x self.out_channels2)
        x = x.view(x.size(0), -1) 
        # we apply our defined fully connected linear layer
        x = self.fc1(x)
        return x

    def _initialize_weights(self):
        # unfortunately not the same as tensorflows truncated normal
        # should be possible to implement an equivalent method
        init.normal(self.conv1.weight, mean=0, std=0.1)
        init.normal(self.conv2.weight, mean=0, std=0.1)

        # in nn.Linear the bias is added -> use 0.1 instead
        init.constant(self.conv1.bias, 0.1)
        init.constant(self.conv2.bias, 0.1)

## Instantiate model and optimizer 

In [4]:
model = CNN()
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-8)

## Training and test method

In [5]:
def train(epoch):
    # indicates training
    # useful for e.g. dropout
    # here not necessary but good practice
    model.train()
    for batch_idx, (data, label) in enumerate(train_loader):
        # input has to be a autograd.Variable
        # allows automatic differentiation
        data, label = Variable(data), Variable(label)
        # has to done before each optimizer step 
        # (each backprop)
        # optimizer accumulates the changes
        optimizer.zero_grad()

        output = model(data)
        loss = F.cross_entropy(output, label)
        # calculates the backpropagation started at the loss
        loss.backward()
        
        # updates the parameters of the network
        # by specified optimizer
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
            
def test():
    # indicates testing
    # useful for e.g. dropout
    # here not necessary but good practice
    model.eval()
    test_loss = 0
    correct = 0
    for data, label in test_loader:
        # volatile true deactivates
        # the calculation of gradients in backward pass
        # also deactivates all nodes that depend on data 
        # means whole network doesn't calculate gradients
        data, label = Variable(data, volatile=True), Variable(label)
        
        output = model(data)
        # sum up batch loss
        test_loss += F.cross_entropy(output, 
                                    label, size_average=False).data[0]  
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1] 
        correct += pred.eq(label.data.view_as(pred)).sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

## Training and evaluation of the model

In [6]:
iterations = 20000
epochs = (iterations * train_batch_size) // mnist_train.__len__()
print("Number of Epochs:", epochs)
for epoch in range(1, epochs+1):
    train(epoch)
    test()

Number of Epochs: 33
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.356297
Train Epoch: 1 [1000/60000 (2%)]	Loss: 1.451460
Train Epoch: 1 [2000/60000 (3%)]	Loss: 0.569053
Train Epoch: 1 [3000/60000 (5%)]	Loss: 0.341912
Train Epoch: 1 [4000/60000 (7%)]	Loss: 0.607321
Train Epoch: 1 [5000/60000 (8%)]	Loss: 0.197221
Train Epoch: 1 [6000/60000 (10%)]	Loss: 0.136129
Train Epoch: 1 [7000/60000 (12%)]	Loss: 0.455817
Train Epoch: 1 [8000/60000 (13%)]	Loss: 0.283751
Train Epoch: 1 [9000/60000 (15%)]	Loss: 0.188153
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.235191
Train Epoch: 1 [11000/60000 (18%)]	Loss: 0.191030
Train Epoch: 1 [12000/60000 (20%)]	Loss: 0.284123
Train Epoch: 1 [13000/60000 (22%)]	Loss: 0.191853
Train Epoch: 1 [14000/60000 (23%)]	Loss: 0.157495
Train Epoch: 1 [15000/60000 (25%)]	Loss: 0.210247
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.102754
Train Epoch: 1 [17000/60000 (28%)]	Loss: 0.069546
Train Epoch: 1 [18000/60000 (30%)]	Loss: 0.091795
Train Epoch: 1 [19000/60000 (32%)]	Loss: 0.11611

Train Epoch: 3 [43000/60000 (72%)]	Loss: 0.010434
Train Epoch: 3 [44000/60000 (73%)]	Loss: 0.102930
Train Epoch: 3 [45000/60000 (75%)]	Loss: 0.014494
Train Epoch: 3 [46000/60000 (77%)]	Loss: 0.047170
Train Epoch: 3 [47000/60000 (78%)]	Loss: 0.012147
Train Epoch: 3 [48000/60000 (80%)]	Loss: 0.035030
Train Epoch: 3 [49000/60000 (82%)]	Loss: 0.039035
Train Epoch: 3 [50000/60000 (83%)]	Loss: 0.104496
Train Epoch: 3 [51000/60000 (85%)]	Loss: 0.007475
Train Epoch: 3 [52000/60000 (87%)]	Loss: 0.033721
Train Epoch: 3 [53000/60000 (88%)]	Loss: 0.017526
Train Epoch: 3 [54000/60000 (90%)]	Loss: 0.025825
Train Epoch: 3 [55000/60000 (92%)]	Loss: 0.041505
Train Epoch: 3 [56000/60000 (93%)]	Loss: 0.006952
Train Epoch: 3 [57000/60000 (95%)]	Loss: 0.112070
Train Epoch: 3 [58000/60000 (97%)]	Loss: 0.037248
Train Epoch: 3 [59000/60000 (98%)]	Loss: 0.028183

Test set: Average loss: 0.0319, Accuracy: 9894/10000 (99%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.002985
Train Epoch: 4 [1000/60000 (2%)]	Loss: 0.008

Train Epoch: 6 [25000/60000 (42%)]	Loss: 0.026382
Train Epoch: 6 [26000/60000 (43%)]	Loss: 0.002717
Train Epoch: 6 [27000/60000 (45%)]	Loss: 0.002338
Train Epoch: 6 [28000/60000 (47%)]	Loss: 0.000178
Train Epoch: 6 [29000/60000 (48%)]	Loss: 0.037868
Train Epoch: 6 [30000/60000 (50%)]	Loss: 0.046977
Train Epoch: 6 [31000/60000 (52%)]	Loss: 0.059902
Train Epoch: 6 [32000/60000 (53%)]	Loss: 0.026711
Train Epoch: 6 [33000/60000 (55%)]	Loss: 0.008044
Train Epoch: 6 [34000/60000 (57%)]	Loss: 0.025398
Train Epoch: 6 [35000/60000 (58%)]	Loss: 0.006396
Train Epoch: 6 [36000/60000 (60%)]	Loss: 0.042800
Train Epoch: 6 [37000/60000 (62%)]	Loss: 0.005714
Train Epoch: 6 [38000/60000 (63%)]	Loss: 0.037611
Train Epoch: 6 [39000/60000 (65%)]	Loss: 0.008317
Train Epoch: 6 [40000/60000 (67%)]	Loss: 0.107686
Train Epoch: 6 [41000/60000 (68%)]	Loss: 0.011409
Train Epoch: 6 [42000/60000 (70%)]	Loss: 0.008468
Train Epoch: 6 [43000/60000 (72%)]	Loss: 0.004173
Train Epoch: 6 [44000/60000 (73%)]	Loss: 0.018097


Train Epoch: 9 [7000/60000 (12%)]	Loss: 0.011595
Train Epoch: 9 [8000/60000 (13%)]	Loss: 0.043196
Train Epoch: 9 [9000/60000 (15%)]	Loss: 0.001507
Train Epoch: 9 [10000/60000 (17%)]	Loss: 0.010153
Train Epoch: 9 [11000/60000 (18%)]	Loss: 0.001826
Train Epoch: 9 [12000/60000 (20%)]	Loss: 0.003445
Train Epoch: 9 [13000/60000 (22%)]	Loss: 0.000646
Train Epoch: 9 [14000/60000 (23%)]	Loss: 0.071327
Train Epoch: 9 [15000/60000 (25%)]	Loss: 0.000234
Train Epoch: 9 [16000/60000 (27%)]	Loss: 0.007921
Train Epoch: 9 [17000/60000 (28%)]	Loss: 0.008831
Train Epoch: 9 [18000/60000 (30%)]	Loss: 0.006009
Train Epoch: 9 [19000/60000 (32%)]	Loss: 0.001673
Train Epoch: 9 [20000/60000 (33%)]	Loss: 0.001750
Train Epoch: 9 [21000/60000 (35%)]	Loss: 0.009714
Train Epoch: 9 [22000/60000 (37%)]	Loss: 0.007639
Train Epoch: 9 [23000/60000 (38%)]	Loss: 0.005793
Train Epoch: 9 [24000/60000 (40%)]	Loss: 0.001754
Train Epoch: 9 [25000/60000 (42%)]	Loss: 0.016961
Train Epoch: 9 [26000/60000 (43%)]	Loss: 0.017158
Tra

Train Epoch: 11 [48000/60000 (80%)]	Loss: 0.000715
Train Epoch: 11 [49000/60000 (82%)]	Loss: 0.000405
Train Epoch: 11 [50000/60000 (83%)]	Loss: 0.003610
Train Epoch: 11 [51000/60000 (85%)]	Loss: 0.006789
Train Epoch: 11 [52000/60000 (87%)]	Loss: 0.005546
Train Epoch: 11 [53000/60000 (88%)]	Loss: 0.000190
Train Epoch: 11 [54000/60000 (90%)]	Loss: 0.000181
Train Epoch: 11 [55000/60000 (92%)]	Loss: 0.011526
Train Epoch: 11 [56000/60000 (93%)]	Loss: 0.000854
Train Epoch: 11 [57000/60000 (95%)]	Loss: 0.001127
Train Epoch: 11 [58000/60000 (97%)]	Loss: 0.000102
Train Epoch: 11 [59000/60000 (98%)]	Loss: 0.001460

Test set: Average loss: 0.0328, Accuracy: 9914/10000 (99%)

Train Epoch: 12 [0/60000 (0%)]	Loss: 0.000112
Train Epoch: 12 [1000/60000 (2%)]	Loss: 0.001334
Train Epoch: 12 [2000/60000 (3%)]	Loss: 0.000710
Train Epoch: 12 [3000/60000 (5%)]	Loss: 0.001654
Train Epoch: 12 [4000/60000 (7%)]	Loss: 0.003771
Train Epoch: 12 [5000/60000 (8%)]	Loss: 0.000991
Train Epoch: 12 [6000/60000 (10%)]	L

...

Train Epoch: 22 [25000/60000 (42%)]	Loss: 0.000022
Train Epoch: 22 [26000/60000 (43%)]	Loss: 0.001611
Train Epoch: 22 [27000/60000 (45%)]	Loss: 0.000050
Train Epoch: 22 [28000/60000 (47%)]	Loss: 0.001768
Train Epoch: 22 [29000/60000 (48%)]	Loss: 0.003313
Train Epoch: 22 [30000/60000 (50%)]	Loss: 0.002334
Train Epoch: 22 [31000/60000 (52%)]	Loss: 0.000190
Train Epoch: 22 [32000/60000 (53%)]	Loss: 0.000390
Train Epoch: 22 [33000/60000 (55%)]	Loss: 0.009133
Train Epoch: 22 [34000/60000 (57%)]	Loss: 0.001090
Train Epoch: 22 [35000/60000 (58%)]	Loss: 0.001960
Train Epoch: 22 [36000/60000 (60%)]	Loss: 0.000456
Train Epoch: 22 [37000/60000 (62%)]	Loss: 0.004089
Train Epoch: 22 [38000/60000 (63%)]	Loss: 0.000074
Train Epoch: 22 [39000/60000 (65%)]	Loss: 0.000154
Train Epoch: 22 [40000/60000 (67%)]	Loss: 0.000551
Train Epoch: 22 [41000/60000 (68%)]	Loss: 0.000070
Train Epoch: 22 [42000/60000 (70%)]	Loss: 0.004819
Train Epoch: 22 [43000/60000 (72%)]	Loss: 0.000457
Train Epoch: 22 [44000/60000 (7

Train Epoch: 25 [3000/60000 (5%)]	Loss: 0.000038
Train Epoch: 25 [4000/60000 (7%)]	Loss: 0.000001
Train Epoch: 25 [5000/60000 (8%)]	Loss: 0.000000
Train Epoch: 25 [6000/60000 (10%)]	Loss: 0.000016
Train Epoch: 25 [7000/60000 (12%)]	Loss: 0.000467
Train Epoch: 25 [8000/60000 (13%)]	Loss: 0.000017
Train Epoch: 25 [9000/60000 (15%)]	Loss: 0.000042
Train Epoch: 25 [10000/60000 (17%)]	Loss: 0.000014
Train Epoch: 25 [11000/60000 (18%)]	Loss: 0.000019
Train Epoch: 25 [12000/60000 (20%)]	Loss: 0.000018
Train Epoch: 25 [13000/60000 (22%)]	Loss: 0.000057
Train Epoch: 25 [14000/60000 (23%)]	Loss: 0.000043
Train Epoch: 25 [15000/60000 (25%)]	Loss: 0.000161
Train Epoch: 25 [16000/60000 (27%)]	Loss: 0.000740
Train Epoch: 25 [17000/60000 (28%)]	Loss: 0.000181
Train Epoch: 25 [18000/60000 (30%)]	Loss: 0.000001
Train Epoch: 25 [19000/60000 (32%)]	Loss: 0.000145
Train Epoch: 25 [20000/60000 (33%)]	Loss: 0.000276
Train Epoch: 25 [21000/60000 (35%)]	Loss: 0.000005
Train Epoch: 25 [22000/60000 (37%)]	Loss:

Train Epoch: 27 [43000/60000 (72%)]	Loss: 0.000010
Train Epoch: 27 [44000/60000 (73%)]	Loss: 0.000007
Train Epoch: 27 [45000/60000 (75%)]	Loss: 0.000006
Train Epoch: 27 [46000/60000 (77%)]	Loss: 0.000019
Train Epoch: 27 [47000/60000 (78%)]	Loss: 0.000011
Train Epoch: 27 [48000/60000 (80%)]	Loss: 0.000042
Train Epoch: 27 [49000/60000 (82%)]	Loss: 0.000064
Train Epoch: 27 [50000/60000 (83%)]	Loss: 0.000035
Train Epoch: 27 [51000/60000 (85%)]	Loss: 0.001277
Train Epoch: 27 [52000/60000 (87%)]	Loss: 0.000000
Train Epoch: 27 [53000/60000 (88%)]	Loss: 0.000062
Train Epoch: 27 [54000/60000 (90%)]	Loss: 0.000005
Train Epoch: 27 [55000/60000 (92%)]	Loss: 0.000008
Train Epoch: 27 [56000/60000 (93%)]	Loss: 0.002144
Train Epoch: 27 [57000/60000 (95%)]	Loss: 0.000200
Train Epoch: 27 [58000/60000 (97%)]	Loss: 0.000264
Train Epoch: 27 [59000/60000 (98%)]	Loss: 0.000109

Test set: Average loss: 0.0450, Accuracy: 9924/10000 (99%)

Train Epoch: 28 [0/60000 (0%)]	Loss: 0.000015
Train Epoch: 28 [1000/6000

Train Epoch: 30 [22000/60000 (37%)]	Loss: 0.000039
Train Epoch: 30 [23000/60000 (38%)]	Loss: 0.000025
Train Epoch: 30 [24000/60000 (40%)]	Loss: 0.000384
Train Epoch: 30 [25000/60000 (42%)]	Loss: 0.000005
Train Epoch: 30 [26000/60000 (43%)]	Loss: 0.002582
Train Epoch: 30 [27000/60000 (45%)]	Loss: 0.000029
Train Epoch: 30 [28000/60000 (47%)]	Loss: 0.000049
Train Epoch: 30 [29000/60000 (48%)]	Loss: 0.000088
Train Epoch: 30 [30000/60000 (50%)]	Loss: 0.000004
Train Epoch: 30 [31000/60000 (52%)]	Loss: 0.000206
Train Epoch: 30 [32000/60000 (53%)]	Loss: 0.000003
Train Epoch: 30 [33000/60000 (55%)]	Loss: 0.003461
Train Epoch: 30 [34000/60000 (57%)]	Loss: 0.000522
Train Epoch: 30 [35000/60000 (58%)]	Loss: 0.000002
Train Epoch: 30 [36000/60000 (60%)]	Loss: 0.000478
Train Epoch: 30 [37000/60000 (62%)]	Loss: 0.000548
Train Epoch: 30 [38000/60000 (63%)]	Loss: 0.000003
Train Epoch: 30 [39000/60000 (65%)]	Loss: 0.000068
Train Epoch: 30 [40000/60000 (67%)]	Loss: 0.000007
Train Epoch: 30 [41000/60000 (6

Train Epoch: 33 [0/60000 (0%)]	Loss: 0.000652
Train Epoch: 33 [1000/60000 (2%)]	Loss: 0.003241
Train Epoch: 33 [2000/60000 (3%)]	Loss: 0.000003
Train Epoch: 33 [3000/60000 (5%)]	Loss: 0.000001
Train Epoch: 33 [4000/60000 (7%)]	Loss: 0.000008
Train Epoch: 33 [5000/60000 (8%)]	Loss: 0.000033
Train Epoch: 33 [6000/60000 (10%)]	Loss: 0.009826
Train Epoch: 33 [7000/60000 (12%)]	Loss: 0.006766
Train Epoch: 33 [8000/60000 (13%)]	Loss: 0.000461
Train Epoch: 33 [9000/60000 (15%)]	Loss: 0.000002
Train Epoch: 33 [10000/60000 (17%)]	Loss: 0.000002
Train Epoch: 33 [11000/60000 (18%)]	Loss: 0.010901
Train Epoch: 33 [12000/60000 (20%)]	Loss: 0.001805
Train Epoch: 33 [13000/60000 (22%)]	Loss: 0.000001
Train Epoch: 33 [14000/60000 (23%)]	Loss: 0.000010
Train Epoch: 33 [15000/60000 (25%)]	Loss: 0.000073
Train Epoch: 33 [16000/60000 (27%)]	Loss: 0.000003
Train Epoch: 33 [17000/60000 (28%)]	Loss: 0.000001
Train Epoch: 33 [18000/60000 (30%)]	Loss: 0.000003
Train Epoch: 33 [19000/60000 (32%)]	Loss: 0.000006